# LangChain: Agents

## Outline:

* Using built in LangChain tools: DuckDuckGo search and Wikipedia
* Defining your own tools

## Built-in LangChain tools

In [1]:
%pip install -qU wikipedia

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip list | findstr "wikipedia"

wikipedia                 1.4.0


In [ ]:
%pip install -qU langchain langchain-core langchain-community
%pip install -qU numexpr

In [ ]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain_ollama import ChatOllama

llm = ChatOllama(model="phi3:3.8b", temperature=0) 

tools = load_tools(["llm-math", "wikipedia"], llm=llm)

agent = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True
)

answer = agent("What is 25% of 300?")
print(answer)



> Entering new AgentExecutor chain...
<think>
Okay, so I need to find 25% of 300. Let me think. Hmm, percentages can sometimes be tricky, but I remember that to find a percentage of a number, you convert the percentage to a decimal and then multiply.

Wait, 25% is the same as 0.25 in decimal. So, 0.25 times 300. Let me do that calculation. 0.25 * 300. Well, 300 divided by 4 is 75, because 25% is a quarter. So 300 divided by 4 equals 75. That makes sense. So 25% of 300 is 75.

But maybe I should use the calculator tool to confirm. Let me check with the Calculator tool. The input would be 25% of 300. Wait, the calculator tool takes a tool_input as a string. So I can input "25% of 300" or maybe just the calculation. Let me see. The calculator's tool_input is a string. So I can write "0.25 * 300" as the input. Let me try that.

Wait, the user said to use the tools if necessary. Since this is a straightforward math problem, maybe the calculator tool can handle it. Let me use the Calculato

ValueError: unknown format from LLM: <think>
Okay, let's tackle this problem. The user wants me to translate the math problem "300 * 0.25" into a Python numexpr expression and then show the output.

First, I need to recall how numexpr works. Numexpr is a Python library that evaluates expressions involving numpy arrays and numbers. But for simple expressions like this, it should handle it directly.

The problem is 300 multiplied by 0.25. In Python, multiplication is *, and 0.25 is a float. So the expression should be straightforward.

Wait, the example given in the problem shows that for "37593 * 67", the numexpr expression is "37593 * 67". So they just write the expression as a string. Similarly, for the exponent, it's "37593**(1/5)".

So for 300 * 0.25, the expression string would be "300 * 0.25". But in Python, the * operator has higher precedence than the * here, but since it's a single multiplication, it's okay.

Wait, but in numexpr, the expressions are evaluated as strings. Let m

In [67]:
# 解决方案1：使用现代LangGraph方式（推荐）
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama
from langchain.tools import Tool
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
import numexpr as ne

# 创建数学计算工具
def calculator(expression: str) -> str:
    """安全地计算数学表达式"""
    try:
        # 清理输入，移除可能的思考标签
        clean_expression = expression.replace("<think>", "").replace("</think>", "").strip()
        result = ne.evaluate(clean_expression)
        return f"计算结果: {result}"
    except Exception as e:
        return f"计算错误: {str(e)}"

# 创建简化的搜索工具
def simple_search(query: str) -> str:
    """简化的搜索工具"""
    try:
        search = DuckDuckGoSearchAPIWrapper()
        return search.run(query)
    except:
        return "搜索功能暂时不可用"

# 定义工具
math_tool = Tool(
    name="calculator",
    description="用于执行数学计算。输入应该是一个有效的数学表达式，例如 '300 * 0.25'",
    func=calculator
)

search_tool = Tool(
    name="search",
    description="用于搜索信息",
    func=simple_search
)

# 初始化模型
llm = ChatOllama(
    model="qwen3:4b",
    temperature=0,
    # 添加系统提示来控制输出格式
    system="你是一个有用的助手。请直接回答问题，不要包含<think>标签或其他格式标记。"
)

# 创建agent
agent = create_react_agent(
    model=llm,
    tools=[math_tool, search_tool],
)

# 运行agent
try:
    result = agent.invoke({
        "messages": [{"role": "user", "content": "What is 25% of 300?"}]
    })
    print("Agent Response:")
    print(result["messages"][-1].content)
except Exception as e:
    print(f"Agent执行出错: {e}")
   


Agent Response:
<think>
Okay, let me go through this step by step. The user asked, "What is 25% of 300?" I called the calculator function with the expression "300 * 0.25" because 25% is equivalent to 0.25 in decimal form. The tool response says the result is 75.0.

First, I need to verify if the calculation is correct. 25% of 300 is indeed 300 multiplied by 0.25. Let me do the math: 300 * 0.25 equals 75. So the tool's response is accurate.

Now, the user probably expects a clear answer in natural language. Since the tool gave 75.0, I should present that as the result. I should state it plainly, like "25% of 300 is 75."

I don't think the search function was needed here because it's a straightforward math problem. The calculator handled it correctly. So the final answer is 75.
</think>

25% of 300 is **75**.




In [68]:
# 运行agent
try:
    result = agent.invoke({
        "messages": [{"role": "user", "content": "鲁迅是谁"}]
    })
    print("Agent Response:")
    print(result["messages"][-1].content)
except Exception as e:
    print(f"Agent执行出错: {e}")

Agent Response:
<think>
好的，用户问“鲁迅是谁”，我之前调用了search工具，但系统返回说搜索功能暂时不可用。现在需要处理这个问题。

首先，用户的问题是关于鲁迅的生平或身份，属于信息查询类。但根据之前的工具设置，search功能不可用，所以不能调用外部搜索。这时候，我需要考虑是否还有其他方法。不过提供的工具里只有calculator和search两个，而search现在不可用，所以可能需要直接回答已知信息。

不过作为AI，我应该知道鲁迅的基本信息，比如他是中国现代文学的重要作家，原名周树人，笔名鲁迅，代表作有《狂人日记》等。虽然用户可能需要更详细的信息，但根据当前工具限制，无法调用搜索，所以应该直接给出答案。

需要检查是否在之前的工具调用中有没有错误。用户之前调用了search，但系统说暂时不可用，所以可能需要跳过工具调用，直接回答。根据问题，用户可能希望得到鲁迅的简介，所以应该用自然语言回答，而不是继续调用工具。

所以，现在应该直接回答用户的问题，说明鲁迅的身份，不需要再调用工具。
</think>

鲁迅（1881年9月25日－1936年10月19日），是中国现代文学的重要作家、思想家和革命家，原名周树人。他以笔名“鲁迅”闻名于世，代表作包括短篇小说《狂人日记》《阿Q正传》《呐喊》《彷徨》等，被誉为“民族魂”。他的作品深刻揭露了封建社会的腐朽和国民劣根性，对中国现代文学和思想启蒙产生了深远影响。


In [24]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"
result = agent(question) 



> Entering new AgentExecutor chain...
Question: What book did Tom M. Mitchell, a professor of Computer Science at CMU, author that discusses AI fundamentals?
Thought: I need to find out which specific work by Professor Mitchell is related to the field of artificial intelligence and his role as Founders University Professor.
Action: 
```json
{
  "action": "wikipedia",
  "action_input": "Tom M. Mitchell"
}
```


d:\project\backend\codespaces-jupyter\.venv\Lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file d:\project\backend\codespaces-jupyter\.venv\Lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')



Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past president of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School of Computer Science at Carnegie Mellon.

Page: Tom Mitchell (Australian footballer)
Summary: Thomas Mitchell (born 31 May 1993) is a professional Australian rules footb

In [ ]:
%pip install -qU langchain-experimental
%pip list | findstr "langchain-experimental"

langchain-experimental    0.3.4


In [36]:
from langchain_experimental.agents.agent_toolkits import create_python_agent
from langchain_experimental.tools.python.tool import PythonREPLTool

agent = create_python_agent(
    llm,
    tool=PythonREPLTool(),
    verbose=True
)

In [38]:
customer_list = [["Harrison", "Chase"], 
                 ["Lang", "Chain"],
                 ["Dolly", "Too"],
                 ["Elle", "Elem"], 
                 ["Geoff","Fusion"], 
                 ["Trance","Former"],
                 ["Jen","Ayai"]
                ]

In [40]:
agent.run(f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}""") 



> Entering new AgentExecutor chain...
Question: Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
Thought: I need to sort the list of customers by their last names and then first names using Python's sorted() function. Then, print out each customer in a formatted way.
Action: [Python_REPL]
Action Input: 
```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))
for customer in sorted_customers:
    print(' '.join(customer))
```
Observation: [Python_REPL] is not a valid tool, try one of [Python_REPL].
Thought:Question: Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['El

ValueError: An output parsing error occurred. In order to pass this error back to the agent and have it try again, pass `handle_parsing_errors=True` to the AgentExecutor. This is the error: Could not parse LLM output: `I apologize for the confusion earlier. Let me correct that and execute the code in Python REPL to get the sorted list of customers by their last names followed by first names. I will then print out each customer's name as a single string with spaces between the last and first names. Here is how we can do it:

```python
customers = [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]
sorted_customers = sorted(customers, key=lambda x: (x[1], x[0]))  # Sort by last name and then first name.
for customer in sorted_customers:
    print(' '.join(customer))
```

Now let's execute this code to get the final answer. I will run it through Python REPL now.`
For troubleshooting, visit: https://pyth

## Define your own tool

In [ ]:
%pip install -qU DateTime

In [43]:
from langchain.agents import tool
from datetime import date

In [44]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [45]:
agent= initialize_agent(
    tools + [time], 
    llm, 
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

**Note**: 

The agent will sometimes come to the wrong conclusion (agents are a work in progress!). 

If it does, please try running it again.

In [46]:
try:
    result = agent("whats the date today?") 
except: 
    print("exception on external access")



> Entering new AgentExecutor chain...
Could not parse LLM output: Question: What is today's date?
Thought: I need to find out what today's date is using a tool that provides current dates without any calculation required on my part. The 'time' function will give me this information directly as it returns the present day when given an empty string input for querying time-related data. 
Action: {"action": "time", "action_input": ""}

For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 
Observation: Invalid or incomplete response
Thought:Could not parse LLM output: Question: What is today's date?
Thought: I need to find out what today's date is using a tool that provides current dates without any calculation required on my part. The 'time' function will give me this information directly as it returns the present day when given an empty string input for querying time-related data. 
Action: {"action": "time", "action_input": ""}

For

In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import JsonOutputParser

message = AIMessage(content='{"foo": "bar"}')
output_parser = JsonOutputParser()
output_parser.invoke(message)

output_parser = JsonOutputParser()
output_parser.invoke(message)

{'foo': 'bar'}

In [ ]:
%pip install -qU langgraph
%pip install -qU langchain-anthropic

In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_ollama import ChatOllama

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# Create the model explicitly
model = ChatOllama(model="qwen3:4b")

agent = create_react_agent(
    model=model,
    tools=[get_weather],
    prompt="You are a helpful assistant"
)

# Run the agent
agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='d2cfe007-b78f-4437-ab54-4184de2a0b1d'),
  AIMessage(content='<think>\nOkay, the user is asking for the weather in SF. SF stands for San Francisco, right? So I need to use the get_weather function. Let me check the tools provided. The function requires a city parameter. The user said "sf", so I should convert that to the full city name maybe? Wait, the function\'s parameters just need the city as a string. Maybe the function can handle "SF" or "San Francisco". But to be safe, I\'ll use "San Francisco" as the city name. Wait, the user wrote "sf" in lowercase. Hmm, but the function probably expects the city name in a standard format. Let me think. Maybe the function is designed to accept city names like "San Francisco" or "SF" as valid inputs. But since the tool\'s example might expect a specific format, but the user just says "sf", I\'ll go with "San Francisco" to be precise. A

In [ ]:
# 解决方案2：使用传统方式但修复格式问题
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain.agents.output_parsers import ReActSingleInputOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.tools.render import render_text_description

# 获取ReAct提示模板
try:
    prompt = hub.pull("hwchase17/react")
except:
    # 如果无法获取hub提示，使用自定义提示
    from langchain.prompts import PromptTemplate
    
    template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}"""

    prompt = PromptTemplate.from_template(template)

# 创建agent
agent = create_react_agent(llm, [math_tool], prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=[math_tool], 
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

try:
    result = agent_executor.invoke({"input": "What is 25% of 300?"})
    print("Traditional Agent Response:")
    print(result["output"])
except Exception as e:
    print(f"传统agent执行出错: {e}")
    print(f"直接计算: 25% of 300 = {300 * 0.25}")

print("\n" + "="*50 + "\n")



d:\project\backend\codespaces-jupyter\.venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  




> Entering new AgentExecutor chain...
<think>
Okay, so I need to find out what 25% of 300 is. Let me think. Hmm, percentages can sometimes be tricky, but I remember that to find a percentage of a number, you convert the percentage to a decimal and then multiply it by the number.

Right, 25% is the same as 0.25 in decimal. So the problem is 0.25 multiplied by 300. Let me check that. If I use the calculator tool provided, I can input the expression '0.25 * 300' to get the result. Wait, the calculator tool takes a math expression as a string. Let me make sure the expression is correct.

Alternatively, maybe I can think of it as 25% of 300 is a quarter of 300, since 25% is a quarter. A quarter of 300 is 75. Let me verify with the calculator. Let's see, if I do 300 * 0.25, that should be 75. Let me use the calculator tool to confirm.

So the action would be calculator with input '300 * 0.25'. Let me input that. The calculator should return 75. Yeah, that makes sense. So the answer is 75.


KeyboardInterrupt: 

In [70]:
# 解决方案3：最简单的直接调用方式
def simple_math_agent(question: str) -> str:
    """简单的数学问题解决器"""
    # 直接使用LLM进行数学计算
    prompt = f"""
请回答以下数学问题: {question}

请按以下格式回答:
1. 首先理解问题
2. 进行计算
3. 给出最终答案

不要使用任何特殊标签或格式。
"""
    
    try:
        response = llm.invoke(prompt)
        return response.content
    except Exception as e:
        # 如果LLM失败，进行直接计算
        if "25%" in question and "300" in question:
            return f"25% of 300 = 300 × 0.25 = {300 * 0.25}"
        return f"计算出错: {e}"

print("Simple Math Agent Response:")
print(simple_math_agent("What is 25% of 300?"))

Simple Math Agent Response:
<think>
首先，用户的问题是：“What is 25% of 300?” 这是中文：“25% of 3.00 是多少？”。我需要按指定的格式回答：1. 首先理解问题；2. 进行计算；3. 给出最终答案。不要使用任何特殊标签或格式。

步骤1：首先理解问题
- 问题是要计算25%的300。意思是求300的25%是多少。
- 25%相当于0.25，所以是求300乘以0.25。

步骤2：进行计算
- 计算300 × 0.25。
- 300 × 0.25 = 75。因为25%是四分之一，300除以4等于75。

步骤3：给出最终答案
- 最终答案是75。

现在，我需要以纯文本格式写回答，不要任何特殊标签或格式。所以，我应该这样写：
1. 首先理解问题：问题要求计算300的25%是多少。
2. 进行计算：300乘以25%等于300 × 0.25 = 75。
3. 给出最终答案：75

确保语言是中文，因为用户的问题是中文的。用户说“请回答以下数学问题”，并且问题用中文写的，所以回答应该用中文。

用户的问题是中文：“What is 25% of 300?” 但问题本身是英文的，不过用户用中文描述了。在回答中，我应该用中文。

回顾用户消息："请回答以下数学问题: What is 25% of 300?" 所以，问题部分是英文的，但用户要求用中文回答？不，用户说“请回答”，但没有指定语言。为了安全，我应该用中文回答，因为用户的问题是用中文写的。

在中文中，数学问题通常用中文描述。所以，我将用中文写回答。

格式：
1. 首先理解问题
2. 进行计算
3. 给出最终答案

不要用任何特殊标签，所以就是纯文本。

草拟回答：
1. 首先理解问题：问题要求计算300的25%是多少。
2. 进行计算：25%等于0.25，因此300乘以0.25等于75。
3. 给出最终答案：75

为了更精确：
- 在计算中，可以写成：300 × 25% = 300 × 0.25 = 75

最终答案是75。

确保没有特殊格式：不要用markdown，不要用**bold**等。

所以，完整回答：
首先理解问题：问题要求计算300的25%是多少。
进行计算：300乘以25%等于75。
给出最终答案：75

在步骤中，要清晰。

用户说